# Contradiction

`Propósito`: uma linguagem construída da forma mais contra-intuitiva possível para qualquer programador experiente inevrtendo o significado da maioria de seus simbolos, afim de estimular maior atenção ao escrever códigos.

## Gramática

Definição geral da linguagem.

```
<program> ::= <definitions> <commands

<definitions> ::= <definition> ||
                        <definition> <definitions>
                        
<definition> ::= int id ;  ||
                        : float id ;

<abstraction> ::= lambda id expression ;

<commands> ::= <command> ||
                    <command> <commands>
                    
<command> ::= id  = <expression> ;

<application> ::= id expression ;

<expression> ::= id ||
                    number ||
                    <expression> <expression> +
                    <expression> <expression> -
                    <expression> <expression> *
                    <expression> <expression> /
```


## Código teste

In [ ]:
bCode = "float x; float y; x = 1; y = x + 2;"
# resulltado esperado :
# x = 1   - dtype int
# y = -1  - dtype int

In [ ]:
mCode = "float x; float y; defmais10(a) : a + 10; x = 1; y = mais10(x) + 2;"
# resulltado esperado :
# x = 1
# y = -11

In [ ]:
hCode = ""
# resulltado esperado :


In [ ]:
prg = "int x; int a; def mais10(y) : y + 10; int a = 10;  x = mais10(a);"

### Analisador Léxico

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MAIN', r'main')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')
lg.add('FOR', r'for')
# lg.add('LAMBDA', r'lambda')
# lg.add("DOT", r'\.')
lg.add('SEMICOL', r';')

lg.add("INDENTATION", r'\:')
lg.add("DEF", r"def")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.ignore('\s+')

lexer = lg.build()

In [ ]:
tokens =lexer.lex(prg)
for token in tokens:
    print(token)

### Árvore sintática

In [ ]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp,expr=None):
        self.id = id
        self.tp = tp
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_vardecl(self)


class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1
    
    def accept(self, visitor):
        visitor.visit_for(self)

class DefFunc(BaseBox):
    def __init__(self, id, param, expr):
        self.id = id
        self.param = param
        self.expr = expr
    
    def accept(self, visitor):
        visitor.visit_deffunc(self)


# class Abstraction(BaseBox):
#       def __init__(self, name,id, expression):
#        self.id = id
#        self.expr = expression
#        self.name = name

#       def accept(self, visitor):
#         visitor.visit_abstraction(self)

# class Application(Expr):
#     def __init__(self, id, expression):
#       self.id = id
#       self.expr = expression

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Func(Expr):
    def __init__(self, id, param):
        self.id = id
        self.param = param

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value
    
    def accept(self, visitor):
      visitor.visit_application(self)

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


### Analisador Sintático

In [ ]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer. indentation
    # 'STRING' "LAMBDA", "DOT"
    ['NUMBER',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'ID','SEMICOL',
     'EQUALS','COMP','IF','ELSE','WHILE','FOR', 'DEF', 'INDENTATION', 'OPEN_PARENS', 'CLOSE_PARENS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)

@pg.production('vardecl : INT ID EQUALS expression SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), p[3])

##################################################
# LAMBDA
##################################################

# @pg.production('vardecl : INT ID EQUALS LAMBDA ID DOT expression SEMICOL')
# def abstraction_int(p):
#     return Abstraction(p[1].getstr(), p[4].getstr(), p[6])

@pg.production('vardecl : DEF ID OPEN_PARENS ID CLOSE_PARENS INDENTATION expression SEMICOL')
def vardecl_function(p):
    return DefFunc(p[1].getstr(), f"_{p[3].getstr()}_", p[6])


##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS openstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('closedstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# Expression
##################################################

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

# @pg.production('expression : ID OPEN_PARENS expression CLOSE_PARENS')
# def expression_lambda(p):
#     return Application(p[0].getstr(), p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : ID OPEN_PARENS expression CLOSE_PARENS')
def expression_func(p):
    return Func(p[0].getstr(), p[2])

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [ ]:
list(lexer.lex(prg))

In [ ]:
arvore = parser.parse(lexer.lex(prg))

In [ ]:
class Visitor(object):
  pass

In [ ]:
# visitor para substituir lambda
lambdas = {}

class GetLambda(Visitor):
    def visit_prog(self, prog):
      prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        pass

    def visit_deffunc(self, ab):
      lambdas[ab.id] = Id(ab.expr)



In [ ]:
code = "int x; int y; x = 1; y = x +1;"
print(code)
tokens = lexer.lex(code)
print(list(tokens))

In [ ]:
arvore = parser.parse(lexer.lex(code))

In [ ]:
arvore.accept(GetLambda())
print(lambdas.keys())

In [ ]:
# visitor para substituir lambda
lambdas = {}

class UnLambda(Visitor):
    def __init__(self, param):
      self.param = param

    def visit_id(self, id):
        id.value = self.param.value

    def visit_number(self, i):
        pass

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"
    

print(lambdas.keys())

### Geração da tabela de simbolos

In [ ]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_deffunc(self, ab):
      ST[ab.id]= "int"

    def visit_vardecl(self, d):
        ST[d.id] = d.tp

### Decorator

In [ ]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):
    def visit_prog(self, i):
        i.decls.accept(self)
        i.stmts.accept(self)


    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_deffunc(self, ab):
      ST[ab.id] = "int"

    def visit_vardecl(self, d):
        ST[d.id]=d.tp

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError(f'id not declared " {i.id} "')
        i.expr.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)

    def visit_for(self, i):
        if i.idinic in ST:
          i.inicdecor_type=ST[i.idinic]
        else:
          raise AssertionError(f'id not declared " {i.value} "')
        i.exprinic.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        if i.idincr in ST:
          i.incrdecor_type=ST[i.idincr]
        else:
          raise AssertionError(f'id not declared " {i.value} "')
        i.exprincr.accept(self)
        i.ie1.accept(self)

    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError(f'id not declared " {i.value} "')

    def visit_func(self, app):
      unlambda = UnLambda(app.param)
      lambdas[app.id].accept(unlambda)

      app.decor_type = "int"

    def visit_number(self, i):
        i.decor_type="int"

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        else :
          a.decor_type="float"

### Verificador de tipos

In [ ]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):
    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_for(self, i):
        # verificação da inicialização do for
        if i.inicdecor_type!=i.exprinic.decor_type:
          raise AssertionError('type error')

        # verificação da comaração do for
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

        # verificação da atualização do for
        if i.incrdecor_type!=i.exprincr.decor_type:
          raise AssertionError('type error')


In [ ]:
class IntermediateCode(Visitor):

  def __init__(self):
      self.level=0  # nível do if-then-else
      self.loop=0  # número do loop
      self.comp={"==":"8","!=":"9","<":"10", "<=":"11", ">":"12", ">=":"13" } #códigos dos comparadores


  def visit_prog(self, i):
      i.stmts.accept(self)

  def visit_statements(self, d):
      d.stmt.accept(self)
      if d.stmts!=None:
            d.stmts.accept(self)

  def visit_statement(self, d):
      d.cmd.accept(self)

  def visit_func(self, app):
      lambdas[app.id].accept(self)

  def visit_atrib(self, i):
      i.expr.accept(self)
      print("sto 0", i.id)

  def visit_ifelse(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        if i.ie2!=None:  # se houver else, gera o código
          i.ie2.accept(self)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_while(self, i):
        self.loop=self.loop+1
        print("loop"+str(self.loop)+":") #gera o rótulo da seção true
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp loop"+str(self.loop)) # pula para o início do loop
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1


  def visit_for(self, i):
        self.loop=self.loop+1
        #gera código para inicialização
        i.exprinic.accept(self)
        print("sto 0",i.idinic)
        print("loop"+str(self.loop)+":") #gera o rótulo inicial do for

        #testa se condição do for é satisfeita
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true

        #gera código para atualização do loop for
        i.exprincr.accept(self)
        print("sto 0",i.idincr)
        print("jmp loop"+str(self.loop)) # pula para o início do loop for

        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1

  def visit_number(self, i):
    print("lit 0 ", i.value)

  def visit_id(self, i):
    print("lod 0 ", i.value)

  def visit_add(self, a):
      a.left.accept(self)
      a.right.accept(self)
      print("opr 0 2")

  def visit_sub(self, a):
    a.left.accept(self)
    a.right.accept(self)
    print("opr 0 3")

  def visit_mul(self, a):
    a.left.accept(self)
    a.right.accept(self)
    print("opr 0 4")

  def visit_div(self, a):
    a.left.accept(self)
    a.right.accept(self)
    print("opr 0 5")

In [ ]:
print(prg)
arvore = parser.parse(lexer.lex(prg))

In [ ]:
arvore=parser.parse(lexer.lex(prg))
arvore.accept(GetLambda())
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())